In [1]:
from utils import *
import cv2


## mask

In [ ]:
mask2d = open_fits_data("dataset/NH2_16p50_0p00_spine.fits")

# skeleton

In [3]:
import skimage.morphology as morph

skeleton = morph.skeletonize(mask2d)

In [ ]:
skeleton.shape

# skeletons instances

In [5]:
import numpy as np
import skimage.morphology as morph
import skimage.measure as measure
import skimage.io as io

In [6]:
labeled_skeleton, num_labels = measure.label(skeleton, return_num=True, connectivity=2)

In [7]:
# Créer une liste pour stocker les images de chaque squelette
skeleton_images = []

# Extraire chaque squelette
for label in range(1, num_labels + 1):
    # Créer une image pour le squelette courant
    individual_skeleton = (labeled_skeleton == label).astype(np.uint8)
    skeleton_images.append(individual_skeleton)

In [ ]:
len(skeleton_images)

In [ ]:
skeleton_images[0].shape

In [ ]:
show_fits_data(skeleton)

In [ ]:
show_fits_data(skeleton_images[20])

# Match mask together

In [12]:
chosen_skeleton = skeleton_images[20]

In [ ]:
data3d_reformated = open_fits_data("dataset/testmin.fits")
image_nan_locs = np.isnan(data3d_reformated)
data3d_reformated[image_nan_locs] = 0

In [ ]:
data3d_reformated.min()

In [ ]:
show_fits_data(data3d_reformated[390,:,:])

In [16]:
mask3D = np.empty(data3d_reformated.shape)

for i in range(data3d_reformated.shape[0]):
    gray_image = data3d_reformated[i, :, :]

    min_value = np.min(gray_image)
    shifted_image = gray_image - min_value
    max_value = np.max(shifted_image)
    normalized_image = shifted_image / max_value

    blurred_image = cv2.GaussianBlur(normalized_image, (5, 5), 1)  # kernel size (5,5) and sigma=1

    temp_image = (blurred_image * 255).astype(np.uint8)
    _, otsu_mask = cv2.threshold(temp_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    otsu_mask = otsu_mask / 255  # Normalize mask to [0,1]

    # Create an elliptical kernel
    elliptical_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    cleaned_mask = cv2.morphologyEx(otsu_mask, cv2.MORPH_OPEN, elliptical_kernel, iterations=1)
    dilated_mask = cv2.dilate(cleaned_mask, elliptical_kernel, iterations=1)

    # Save the resulting mask
    mask3D[i, :, :] = dilated_mask


In [ ]:
from matplotlib.colors import ListedColormap
 
i = 390

# Create empty RGB arrays for the colored images
height, width = mask2d.shape
color_image1 = np.zeros((height, width, 3))  # For pink
color_image2 = np.zeros((height, width, 3))  # For green
color_image2 = np.zeros((height, width, 3))  # For green

# Color the first image (pink)
color_image1[chosen_skeleton.astype(bool)] = [1, 0.75, 0.8]  # RGB for pink

# Color the second image (green)
color_image2[mask3D[i,:,:].astype(bool)] = [0, 1, 0]  # RGB for green

# Blend the two images by adding the RGB values and clipping them to 1 (since RGB values range from 0 to 1)
blended_image = color_image1 + color_image2
blended_image = np.clip(blended_image, 0, 1)  # Ensures no values go over 1

# Create a figure and display the images
plt.figure(figsize=(20, 15))

plt.subplot(1, 4, 1)
plt.title('Skeleton (Pink)')
plt.imshow(color_image1)
plt.axis('off')

plt.subplot(1, 4, 2)
plt.title('Data 3d sur i')
plt.imshow(data3d_reformated[i,:,:])
plt.axis('off')

plt.subplot(1, 4, 3)
plt.title('Mask data 3d(Green) sur i')
plt.imshow(color_image2)
plt.axis('off')

plt.subplot(1, 4, 4)
plt.title('Superposed Image')
plt.imshow(blended_image)
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
data3d_reformated.shape

# Option 1

In [24]:
y_coords, x_coords = np.where(chosen_skeleton == 1)
# Combiner les coordonnées dans un tableau ou une liste
coordinates = list(zip(x_coords, y_coords))

# Afficher les coordonnées
for coord in coordinates:
    print(coord)

(np.int64(66), np.int64(201))
(np.int64(67), np.int64(201))
(np.int64(68), np.int64(201))
(np.int64(69), np.int64(201))
(np.int64(70), np.int64(201))
(np.int64(64), np.int64(202))
(np.int64(65), np.int64(202))
(np.int64(71), np.int64(202))
(np.int64(54), np.int64(203))
(np.int64(62), np.int64(203))
(np.int64(63), np.int64(203))
(np.int64(71), np.int64(203))
(np.int64(72), np.int64(203))
(np.int64(73), np.int64(203))
(np.int64(74), np.int64(203))
(np.int64(54), np.int64(204))
(np.int64(61), np.int64(204))
(np.int64(70), np.int64(204))
(np.int64(54), np.int64(205))
(np.int64(61), np.int64(205))
(np.int64(55), np.int64(206))
(np.int64(56), np.int64(206))
(np.int64(57), np.int64(206))
(np.int64(58), np.int64(206))
(np.int64(60), np.int64(206))
(np.int64(59), np.int64(207))
(np.int64(60), np.int64(207))
(np.int64(61), np.int64(208))
(np.int64(61), np.int64(209))
(np.int64(61), np.int64(210))
(np.int64(61), np.int64(211))
(np.int64(61), np.int64(212))
(np.int64(61), np.int64(213))
(np.int64(

In [1]:
import sys
!{sys.executable} -m pip install seaborn


  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
